# 1.  Introduction & Problem Definition  

## CFPB Consumer Complaints EDA

## 📦 About the Dataset

The Consumer Financial Protection Bureau (CFPB) collects and publishes thousands of consumer complaints each week regarding financial products and services.  
Each complaint is sent to the relevant company, and responses are tracked. By making this data public, the CFPB aims to improve transparency and accountability in the financial marketplace.

- **Source:** [CFPB Consumer Complaints Database](https://www.consumerfinance.gov/data-research/consumer-complaints/)
- **Data columns:**  
    - Date received, product, sub-product, issue, sub-issue, company, state, response, etc.
    - Includes text columns, categorical columns, and timestamps
- **Typical size:** 2M+ rows

---

## 🎯 Project Goal

**To analyze the landscape of consumer complaints in the U.S. financial sector and extract actionable insights for companies, regulators, and consumers.**

### Main Objectives
- Understand trends in complaints across time, products, and companies
- Identify major issues and pain points faced by consumers
- Assess company response rates and behaviors

---

## 🛠️ What We Do (Methods)

- **Data Cleaning & Preprocessing**
    - Handling missing/null values
    - Standardizing product and issue names
    - Parsing dates and categoricals
- **Exploratory Data Analysis (EDA)**
    - Trends over time: volume of complaints per month/year
    - Breakdown by product, sub-product, issue, state, and company
    - Distribution of company responses (timeliness, public statements)
    - Visualization: bar charts, time series, heatmaps, etc.
- **Business Insights**
    - Detect which products/companies have the most issues
    - Find common root causes of complaints
    - Compare top-performing and underperforming companies

---

## 🔍 Key Questions & Insights

- Which financial products generate the most consumer complaints?
- What are the most common issues and sub-issues?
- Which companies have the highest volume of complaints?
- How do response rates and response types differ across companies?
- Are there geographic (state-level) patterns in complaint volume or issues?
- Has the complaint volume increased/decreased over time?

---

## 📊 Possible KPIs (Key Performance Indicators)

- **Total Complaints per Product/Company**
- **Percentage of Timely Responses**
- **Average Response Time** (if timestamp data is available)
- **Complaint Resolution Rate** (if status available)
- **Top Issues by Frequency**
- **Null/Missing Data Rate** per critical column

---

## 💡 Expected Outcomes

- **Actionable recommendations** for financial companies to improve service
- **Transparency** for consumers on which companies/products have most complaints
- **Regulatory insights** to help prioritize oversight and policy

---

## 📝 Author

Minhyeok Son  
[LinkedIn](hwww.linkedin.com/in/minhyeokson) | [GitHub](https://github.com/Shawn-Son)

---

# 2. Initial Data Exploration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../../Datasets/finance/consumer_complaints.csv')

/var/folders/vs/dy5n5y354mz0k83z44gbr5ph0000gn/T/ipykernel_235/182790902.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../Datasets/finance/consumer_complaints.csv')


## 1. Basic Structure & Info

In [ ]:
df.head()

# Benefit:
# 	•	Quickly spot obvious issues (e.g., wrong separators, extra unnamed columns, messy text formats)
# 	•	Gives you context on what the dataset really looks like before running big analyses

# Analysis:
#   • 18 columns total, including categorical, text, date, and ID fields
#   • Several columns (e.g., sub_product, consumer_complaint_narrative, company_public_response, tags) contain NaN values
#   • Text-heavy fields like "issue" may require NLP-specific cleaning
#   • Date fields (date_received, date_sent_to_company) seem to be in MM/DD/YYYY format — will need to be parsed
#   • "product" appears to have repeated categories, suggesting grouping and standardization may be needed
#   • "complaint_id" is numeric and unique, useful as a primary key

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [ ]:
df.shape

# Benefit:
# 	•	Tells you immediately if you’re dealing with a small, medium, or massive dataset (important for deciding on sampling, optimization, or full processing)
# 	•	Helps you cross-check with expected size from documentation

(555957, 18)

In [ ]:
df.info()

# Benefit:
# 	•	Reveals if columns are stored in wrong type (e.g., dates as strings, numbers as objects)
# 	•	Highlights missing value patterns early, so you can decide whether to clean, drop, or impute
# 	•	Avoids future errors (e.g., trying to do numeric calculations on a string column)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 555957 entries, 0 to 555956
Data columns (total 18 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   date_received                 555957 non-null  object
 1   product                       555957 non-null  object
 2   sub_product                   397635 non-null  object
 3   issue                         555957 non-null  object
 4   sub_issue                     212622 non-null  object
 5   consumer_complaint_narrative  66806 non-null   object
 6   company_public_response       85124 non-null   object
 7   company                       555957 non-null  object
 8   state                         551070 non-null  object
 9   zipcode                       551452 non-null  object
 10  tags                          77959 non-null   object
 11  consumer_consent_provided     123458 non-null  object
 12  submitted_via                 555957 non-null  object
 13 

In [ ]:
df.columns.to_list()

# Benefit:
# 	•	Lets you check for typos, duplicates, or spaces in column names (which cause coding errors later)
# 	•	Useful for renaming columns early in the workflow for consistency

['date_received',
 'product',
 'sub_product',
 'issue',
 'sub_issue',
 'consumer_complaint_narrative',
 'company_public_response',
 'company',
 'state',
 'zipcode',
 'tags',
 'consumer_consent_provided',
 'submitted_via',
 'date_sent_to_company',
 'company_response_to_consumer',
 'timely_response',
 'consumer_disputed?',
 'complaint_id']

## 2. Missing Value Overview

In [ ]:
df.isnull().sum().sort_values(ascending=False)

# Benefit:
# 	•	Identifies data quality issues early → You can see which columns have missing values and how severe the problem is.
# 	•	Helps decide data cleaning strategies → You can choose whether to drop the column, fill missing values (imputation), or investigate why they are missing.
# 	•	Prevents analysis errors → Many functions and visualizations break or give wrong results when missing values are present.
# 	•	Improves model performance later → Machine learning algorithms can’t handle NaNs by default, so spotting them now avoids pipeline errors later.
# 	•	Saves time → You focus preprocessing only where it’s needed instead of blindly cleaning all columns.

consumer_complaint_narrative    489151
tags                            477998
company_public_response         470833
consumer_consent_provided       432499
sub_issue                       343335
sub_product                     158322
state                             4887
zipcode                           4505
date_sent_to_company                 0
consumer_disputed?                   0
timely_response                      0
company_response_to_consumer         0
date_received                        0
submitted_via                        0
product                              0
company                              0
issue                                0
complaint_id                         0
dtype: int64

## 3. Unique Values & Cardinality

In [ ]:
df.nunique().sort_values(ascending=False)

# Benefit:
#   •  Shows how many unique values each column has (cardinality)
#   •  Helps identify high-cardinality columns in categorical data
#        → Example: 'company' might have 10,000+ unique values — too many for certain plots or models
#   •  Detects binary columns stored as text (e.g., "Yes"/"No"), which you might convert to True/False
#   •  Finds potential data quality issues:
#        → Typos, inconsistent casing ("Mortgage" vs "mortgage"), trailing spaces
#   •  Highlights ID-like columns (e.g., 'complaint_id') that should not be used in analysis
#   •  Guides preprocessing decisions:
#        → Low unique count → treat as categorical
#        → Very high unique count → consider grouping or excluding
#   •	Low cardinality columns → good for grouping, bar plots, categorical encoding.
#   •	High cardinality columns → not good for categorical plots, may slow down models, might need aggregation or removal.

complaint_id                    555957
consumer_complaint_narrative     65646
zipcode                          27052
company                           3605
date_received                     1608
date_sent_to_company              1557
issue                               95
sub_issue                           68
state                               62
sub_product                         46
product                             11
company_public_response             10
company_response_to_consumer         8
submitted_via                        6
consumer_consent_provided            4
tags                                 3
timely_response                      2
consumer_disputed?                   2
dtype: int64

## 4. Categorical Feature Summary

In [10]:
categorical_cols = df.select_dtypes(include='object').columns
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False).head(5))


Column: date_received
date_received
08/27/2015    963
06/26/2014    916
08/26/2015    912
07/08/2015    827
08/12/2015    819
Name: count, dtype: int64

Column: product
product
Mortgage                   186475
Debt collection            101052
Credit reporting            91854
Credit card                 66468
Bank account or service     62563
Name: count, dtype: int64

Column: sub_product
sub_product
NaN                                      158322
Other mortgage                            74319
Conventional fixed mortgage               57182
Checking account                          44369
Other (i.e. phone, health club, etc.)     29617
Name: count, dtype: int64

Column: issue
issue
Loan modification,collection,foreclosure    97191
Incorrect information on credit report      66718
Loan servicing, payments, escrow account    60375
Cont'd attempts collect debt not owed       42285
Account opening, closing, or management     26661
Name: count, dtype: int64

Column: sub_issue
sub_issue
N

## 5. Text Columns Check

In [11]:
df['consumer_complaint_narrative'].dropna().apply(len).describe()

count    66806.000000
mean      1039.587327
std        910.270430
min         10.000000
25%        391.000000
50%        740.000000
75%       1383.000000
max       5153.000000
Name: consumer_complaint_narrative, dtype: float64

## 6. Date Columns Check

In [12]:
df['date_received'] = pd.to_datetime(df['date_received'], errors='coerce')
df['date_sent_to_company'] = pd.to_datetime(df['date_sent_to_company'], errors='coerce')

print(df['date_received'].min(), df['date_received'].max())

2011-12-01 00:00:00 2016-04-25 00:00:00


## 7. Initial Decriptive Stats

In [13]:
df.describe(include='all').transpose()

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
date_received,555957,NaN,NaN,NaN,2014-06-28 00:45:51.648778496,2011-12-01 00:00:00,2013-08-09 00:00:00,2014-08-07 00:00:00,2015-06-28 00:00:00,2016-04-25 00:00:00,NaN
product,555957,11,Mortgage,186475,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_product,397635,46,Other mortgage,74319,NaN,NaN,NaN,NaN,NaN,NaN,NaN
issue,555957,95,"Loan modification,collection,foreclosure",97191,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sub_issue,212622,68,Account status,26798,NaN,NaN,NaN,NaN,NaN,NaN,NaN
consumer_complaint_narrative,66806,65646,This company continues to report on my credit ...,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
company_public_response,85124,10,Company chooses not to provide a public response,52478,NaN,NaN,NaN,NaN,NaN,NaN,NaN
company,555957,3605,Bank of America,55998,NaN,NaN,NaN,NaN,NaN,NaN,NaN
state,551070,62,CA,81700,NaN,NaN,NaN,NaN,NaN,NaN,NaN
zipcode,551452,27052,300XX,1205,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Data Preprocessing 

# 4. Exploratory Data Analysis

# 5. KPI Calculation & Summary

# 6. Insights & Recommendations 